<hr>

# <center> Sustainability Report Analyzer  </center>

<hr>

In [1]:
import os

In [2]:
from dotenv import load_dotenv, find_dotenv


load_dotenv(find_dotenv(r"C:\AIDTM\Trimester - 2\GenAI\Lab\dotenvKey.env"))

True

In [41]:
from IPython.display import Markdown, display

In [3]:
from langchain_classic.chains import LLMChain, RetrievalQA

In [4]:
from langchain_classic.prompts import PromptTemplate

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [6]:
from langchain_community.vectorstores import Chroma

In [19]:
from langchain_community.vectorstores import FAISS

In [7]:
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings

In [10]:
from langchain_community.document_loaders import PyPDFLoader

# Example usage
loader = PyPDFLoader(r"C:\Users\Admin\Downloads\Tata Steel Ltd - 2024 - Business Responsibility and Sustainability Report (SustainabilityReports.com).pdf")
docs = loader.load()

# docs will be a list of Document objects

In [11]:
docs

[Document(metadata={'producer': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'creator': 'Adobe InDesign 19.4 (Windows)', 'creationdate': '2024-07-18T19:19:24+05:30', 'moddate': '2025-11-28T16:55:53+00:00', 'source': 'C:\\Users\\Admin\\Downloads\\Tata Steel Ltd - 2024 - Business Responsibility and Sustainability Report (SustainabilityReports.com).pdf', 'total_pages': 80, 'page': 0, 'page_label': '1'}, page_content='BUSINESS RESPONSIBILITY AND SUSTAINABILITY REPORT\nFinancial Year 2023-2024\nIndex\nSECTION A: GENERAL DISCLOSURES 147\nSECTION B: MANAGEMENT AND PROCESS DISCLOSURES 167\nSECTION C: PRINCIPLE WISE PERFORMANCE DISCLOSURES 172\nPrinciple 1:  Businesses should conduct and govern themselves with integrity, and in a manner that is \nethical, transparent and accountable\n172\nPrinciple 2:  Businesses should provide goods and services in a manner that is sustainable and safe 176\nPrinciple 3:  Businesses should respect and promote the well-being of all employees, including those 

In [34]:
docs[9].metadata
# {'source': 'report.pdf', 'page': 9}


{'producer': 'pdf-lib (https://github.com/Hopding/pdf-lib)',
 'creator': 'Adobe InDesign 19.4 (Windows)',
 'creationdate': '2024-07-18T19:19:24+05:30',
 'moddate': '2025-11-28T16:55:53+00:00',
 'source': 'C:\\Users\\Admin\\Downloads\\Tata Steel Ltd - 2024 - Business Responsibility and Sustainability Report (SustainabilityReports.com).pdf',
 'total_pages': 80,
 'page': 9,
 'page_label': '10'}

In [14]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 700,chunk_overlap = 150)

chunks = text_splitter.split_documents(docs)

print(f'Total Chunks Created: {len(chunks)}')

Total Chunks Created: 563


In [15]:
chunks

[Document(metadata={'producer': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'creator': 'Adobe InDesign 19.4 (Windows)', 'creationdate': '2024-07-18T19:19:24+05:30', 'moddate': '2025-11-28T16:55:53+00:00', 'source': 'C:\\Users\\Admin\\Downloads\\Tata Steel Ltd - 2024 - Business Responsibility and Sustainability Report (SustainabilityReports.com).pdf', 'total_pages': 80, 'page': 0, 'page_label': '1'}, page_content='BUSINESS RESPONSIBILITY AND SUSTAINABILITY REPORT\nFinancial Year 2023-2024\nIndex\nSECTION A: GENERAL DISCLOSURES 147\nSECTION B: MANAGEMENT AND PROCESS DISCLOSURES 167\nSECTION C: PRINCIPLE WISE PERFORMANCE DISCLOSURES 172\nPrinciple 1:  Businesses should conduct and govern themselves with integrity, and in a manner that is \nethical, transparent and accountable\n172\nPrinciple 2:  Businesses should provide goods and services in a manner that is sustainable and safe 176\nPrinciple 3:  Businesses should respect and promote the well-being of all employees, including those 

In [33]:
chunks[9].metadata
# {'source': 'report.pdf', 'page': 9}


{'producer': 'pdf-lib (https://github.com/Hopding/pdf-lib)',
 'creator': 'Adobe InDesign 19.4 (Windows)',
 'creationdate': '2024-07-18T19:19:24+05:30',
 'moddate': '2025-11-28T16:55:53+00:00',
 'source': 'C:\\Users\\Admin\\Downloads\\Tata Steel Ltd - 2024 - Business Responsibility and Sustainability Report (SustainabilityReports.com).pdf',
 'total_pages': 80,
 'page': 2,
 'page_label': '3'}

In [16]:
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

In [36]:
db = FAISS.from_documents(chunks, embeddings)


In [37]:
retri = db.as_retriever()

In [57]:
template = """
You are a Corporate Sustainability Insights Assistant.
Your task is to extract and summarize information strictly from the organization's Sustainability / ESG / BRSR report.

Context:
{context}

User Question:
{question}

Response Requirements:
1. Start the output with:
   - A **H1 title** auto-generated based on the question (e.g., "# Top Sustainability Initiatives").
   - A **H2 subtitle** providing a one-line executive summary of the response.
2. Provide the answer in **corporate-style bullet points**, each followed by an explanation of **40–50 words**.
3. Each bullet point must:
   - Be factual, action-oriented, and derived strictly from the provided context.
   - Reflect an executive-level sustainability communication tone.
   - Use strong corporate verbs like "Implemented", "Strengthened", "Enhanced", "Expanded", "Achieved".
4. Base the answer *only* on the given context — do NOT infer or hallucinate.
5. If the question requires prioritization (e.g., “Top 3 initiatives”), pick the most significant items mentioned.
6. If partial information exists, provide only the supported points.
7. If no relevant information is found, respond exactly:
   "Sorry, I don't know. Kindly contact the company number."
8. Use **Markdown formatting** for headings and bullet points.
9. Include page references where available using *(Source: Page X)*.

Tone Guidelines:
- Executive-level
- Crisp, clear, and outcome-focused
- No fluff or repetition

Output Format:
- One H1 title
- One H2 subtitle (one line)
- Bullet points with 40–50-word explanations
"""




qa_prompt = PromptTemplate(
    input_variables = ["context","question"],
    template = template
)

In [39]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    model = "gemini-2.5-pro",
    model_provider = "google_genai", 
    api_key=os.environ["Google_API_Key"]  
)

In [58]:
qa_chain = RetrievalQA.from_chain_type(
    llm= llm,
    retriever = retri,
    chain_type="stuff",
    return_source_documents = False,
    chain_type_kwargs = {'prompt':qa_prompt}
)

In [59]:
result = qa_chain.invoke("give summary of tata steel report ")
print(result)

{'query': 'give summary of tata steel report ', 'result': "# Summary of Tata Steel's Sustainability Report\n## This summary highlights Tata Steel's commitment to Net Zero emissions, its management of an extensive value chain, and transparent stakeholder communication.\n\n*   **Established a Net Zero Emissions Target**\n    The company has committed to achieving Net Zero across all operations by 2045, acknowledging its substantial emission footprint. This goal is supported by a detailed Climate Change Report aligned with TCFD recommendations, outlining its strategy, governance, risk management, metrics, and specific targets. *(Source: Page 224)*\n\n*   **Assessed Environmental Impact of the Value Chain**\n    Tata Steel has evaluated its long and integrated value chain, which handles over 100 million tonnes of material annually. The company recognizes the negative environmental impacts, including greenhouse gas emissions, from the production, transportation, and use of materials in its 

In [60]:
# If result is a dict like {"query": "...", "result": "..."}
display(Markdown(result["result"]))

# Summary of Tata Steel's Sustainability Report
## This summary highlights Tata Steel's commitment to Net Zero emissions, its management of an extensive value chain, and transparent stakeholder communication.

*   **Established a Net Zero Emissions Target**
    The company has committed to achieving Net Zero across all operations by 2045, acknowledging its substantial emission footprint. This goal is supported by a detailed Climate Change Report aligned with TCFD recommendations, outlining its strategy, governance, risk management, metrics, and specific targets. *(Source: Page 224)*

*   **Assessed Environmental Impact of the Value Chain**
    Tata Steel has evaluated its long and integrated value chain, which handles over 100 million tonnes of material annually. The company recognizes the negative environmental impacts, including greenhouse gas emissions, from the production, transportation, and use of materials in its steelmaking processes. *(Source: Page 224)*

*   **Maintained Transparent Stakeholder Communication**
    The company ensures transparent engagement with its stakeholders. This includes proactively informing customers of major disruptions through its website and social media, and providing a clear breakdown of domestic versus international sales to ensure transparency in its global commercial operations. *(Source: Page 150, 224)*